## Imports

In [52]:
from utils.data_utils import load_anndata
from train import load_config, create_model
from models.babel import BabelModel
import json
import yaml
import scanpy as sc
import pandas as pd
import numpy as np

from argparse import Namespace
from itertools import cycle

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import seaborn as sns
import sklearn
import torch
from pytorch_lightning import LightningModule
from sklearn.decomposition import PCA
from sklearn.metrics import (
    adjusted_rand_score,
    auc,
    normalized_mutual_info_score,
    roc_curve,
    silhouette_score,
)
from sklearn.preprocessing import label_binarize
from torch import nn
from torch.nn import functional as F

from models.building_blocks import Block

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

import scanpy as sc
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from lightning.pytorch.loggers import TensorBoardLogger
import gdown

## Load data

In [53]:
test_data = load_anndata(
    mode="test",
    normalize=True,
    remove_batch_effect=False,
    target_hierarchy_level=-1,
    preload_subsample_frac=None,
)

c:\Users\adamb\miniconda3\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [54]:
hierarchy = {'level1': ['Hematopoietic Stem Cells and Progenitors', 'Hematopoietic Stem Cells and Progenitors', 'Hematopoietic Stem Cells and Progenitors', 'Hematopoietic Stem Cells and Progenitors', 'Monocytes and Dendritic Cells', 'Monocytes and Dendritic Cells', 'Monocytes and Dendritic Cells', 'Monocytes and Dendritic Cells', 'Monocytes and Dendritic Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'T Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'B Cells and Plasma Cells', 'Natural Killer Cells', 'Natural Killer Cells', 'Erythroid Lineage', 'Erythroid Lineage', 'Erythroid Lineage', 'Erythroid Lineage', 'Innate Lymphoid Cells (ILCs)', 'Innate Lymphoid Cells (ILCs)'], 'level2': ['HSC', 'Progenitors', 'Progenitors', 'Progenitors', 'Monocytes', 'Monocytes', 'Dendritic Cells', 'Dendritic Cells', 'Dendritic Cells', 'CD4+ T Cells', 'CD4+ T Cells', 'CD4+ T Cells', 'CD4+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'CD8+ T Cells', 'Special T Cell Types', 'Special T Cell Types', 'Special T Cell Types', 'Special T Cell Types', 'Special T Cell Types', 'Special T Cell Types', 'Naive B Cells', 'Naive B Cells', 'Transitional B Cells', 'B1 B Cells', 'B1 B Cells', 'Plasma Cells', 'Plasma Cells', 'Plasmablasts', 'Plasmablasts', 'NK', 'NK CD158e1+', 'Erythroblasts and Precursors', 'Erythroblasts and Precursors', 'Erythroblasts and Precursors', 'Reticulocytes', 'ILC', 'ILC1'], 'level3': ['HSC', 'Lymph prog', 'G/M prog', 'MK/E prog', 'CD14+ Mono', 'CD16+ Mono', 'cDC1', 'cDC2', 'pDC', 'CD4+ T naive', 'CD4+ T activated', 'CD4+ T activated integrinB7+', 'CD4+ T CD314+ CD45RA+', 'CD8+ T naive', 'CD8+ T CD57+ CD45RO+', 'CD8+ T CD57+ CD45RA+', 'CD8+ T TIGIT+ CD45RO+', 'CD8+ T TIGIT+ CD45RA+', 'CD8+ T CD49f+', 'CD8+ T CD69+ CD45RO+', 'CD8+ T CD69+ CD45RA+', 'CD8+ T naive CD127+ CD26- CD101-', 'MAIT', 'gdT CD158b+', 'gdT TCRVD2+', 'dnT', 'T reg', 'T prog cycling', 'Naive CD20+ B IGKC+', 'Naive CD20+ B IGKC-', 'Transitional B', 'B1 B IGKC+', 'B1 B IGKC-', 'Plasma cell IGKC+', 'Plasma cell IGKC-', 'Plasmablast IGKC+', 'Plasmablast IGKC-', 'NK', 'NK CD158e1+', 'Proerythroblast', 'Erythroblast', 'Normoblast', 'Reticulocyte', 'ILC', 'ILC1'], 'description': ['Hematopoietic Stem Cells', 'Lymphoid progenitors', 'Granulocyte-Macrophage progenitors', 'Megakaryocyte-Erythroid progenitors', 'CD14+ Monocytes', 'CD16+ Monocytes', 'Conventional Dendritic Cells, subtype 1', 'Conventional Dendritic Cells, subtype 2', 'Plasmacytoid Dendritic Cells', 'Naive CD4+ T Cells', 'Activated CD4+ T Cells', 'Activated CD4+ T Cells expressing integrin β7', 'CD4+ T Cells expressing CD314 and CD45RA', 'Naive CD8+ T Cells', 'CD8+ T Cells expressing CD57 and CD45RO', 'CD8+ T Cells expressing CD57 and CD45RA', 'CD8+ T Cells expressing TIGIT and CD45RO', 'CD8+ T Cells expressing TIGIT and CD45RA', 'CD8+ T Cells expressing CD49f', 'CD8+ T Cells expressing CD69 and CD45RO', 'CD8+ T Cells expressing CD69 and CD45RA', 'Naive CD8+ T Cells expressing CD127, not CD26 or CD101', 'Mucosal-Associated Invariant T Cells', 'Gamma Delta T Cells expressing CD158b', 'Gamma Delta T Cells expressing TCRVδ2', 'Double-Negative T Cells', 'Regulatory T Cells', 'Cycling Progenitor T Cells', 'Naive B Cells expressing CD20 and IGKC', 'Naive B Cells expressing CD20, not IGKC', 'Transitional B Cells', 'B1 B Cells expressing IGKC', 'B1 B Cells not expressing IGKC', 'Plasma Cells expressing IGKC', 'Plasma Cells not expressing IGKC', 'Plasmablasts expressing IGKC', 'Plasmablasts not expressing IGKC', 'Natural Killer Cells', 'Natural Killer Cells expressing CD158e1', 'Proerythroblast', 'Erythroblast', 'Normoblast', 'Reticulocyte', 'Innate Lymphoid Cells', 'ILC1']}
hierarchy_df = pd.DataFrame(data = hierarchy)
test_data.obs = pd.merge(test_data.obs, hierarchy_df, left_on="cell_type", right_on="level3", how="left")

c:\Users\adamb\miniconda3\Lib\site-packages\anndata\_core\anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


## Load Latent

In [56]:
LATENT_NAME = "latent_omiae"
DATA_PATH = "../data/latent/latent_omiae_test"

In [1]:
LATENT_NAME = "omivae_latent"
DATA_PATH = "../data/latent/omivae_2024-06-15_14-02-51/latent_test.zip"

In [16]:
LATENT_NAME = "babel_latent_adt"
DATA_PATH = "../data/latent/adam_babel/adam_babel_latent_test_adt"

In [34]:
LATENT_NAME = "babel_latent_gex"
DATA_PATH = "../data/latent/adam_babel/adam_babel_latent_test_gex"

In [57]:
LATENT_NAME = "latent_omiae"
DATA_PATH = "../data/latent/omiae_simple_2024-06-15_10-32-21/latent_omiae_test.zip"

In [58]:
test_data.obsm[LATENT_NAME] = torch.load(DATA_PATH)

## Downstream task

In [67]:
LATENT_DIM = 16
DATA = test_data
LEVEL = 'level3'
HIDDEN_DIM = 16 * 4

In [68]:
# Extract the labels (cell types)
labels = DATA.obs[LEVEL].values

# Encode labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert to PyTorch tensors
X = torch.tensor(DATA.obsm[LATENT_NAME], dtype=torch.float32)
y = torch.tensor(labels_encoded, dtype=torch.long)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

C:\Users\adamb\AppData\Local\Temp\ipykernel_26028\2057320055.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(DATA.obsm[LATENT_NAME], dtype=torch.float32)


In [69]:
from models.building_blocks import Block


class ClassificationModel(pl.LightningModule):
    def __init__(self, latent_dim, num_classes, hidden_dim, do_batch_norm=False):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes),
        )
        self.loss = nn.CrossEntropyLoss()  # Adjust

    def forward(self, z):
        return self.layers(z)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log(
            "train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log(
            "val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True
        )
        
        return loss

    def configure_optimizers(self):
        # Define optimizer here
        optimizer = torch.optim.Adam(
            self.parameters(), lr=0.001
        )  # Adjust learning rate
        return optimizer

In [70]:
output_dim = len(np.unique(labels_encoded))

model = ClassificationModel(latent_dim=LATENT_DIM,
                            num_classes=output_dim,
                            hidden_dim=HIDDEN_DIM,
                            do_batch_norm=False)

# Define a trainer
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=f'logs/classification_checkpoints/{LATENT_NAME}',
    filename='model-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min',
)
logger = TensorBoardLogger("./logs", name=f"{LATENT_NAME}_classifing_logs")
trainer = pl.Trainer(max_epochs=10, callbacks=[checkpoint_callback])

# Train the model
trainer.fit(model, train_loader, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type             | Params
--------------------------------------------
0 | layers | Sequential       | 3.7 K 
1 | loss   | CrossEntropyLoss | 0     
--------------------------------------------
3.7 K     Trainable params
0         Non-trainable params
3.7 K     Total params
0.015     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\adamb\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\adamb\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [71]:
# Loading the best model for evaluation
best_model_path = checkpoint_callback.best_model_path
best_model = ClassificationModel.load_from_checkpoint(
    best_model_path,
    latent_dim=LATENT_DIM,
    num_classes=output_dim,
    hidden_dim=HIDDEN_DIM,
    do_batch_norm=False).cpu()

# Evaluate the model on test set
best_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = best_model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 77.64%


In [72]:
output = best_model(X_test)
_, predicted = torch.max(output.data, 1)

In [73]:
def print_metrics(y_true, y_pred):
    balanced_accuracy_score = sklearn.metrics.balanced_accuracy_score(y_true, y_pred)
    accuracy_score = sklearn.metrics.accuracy_score(y_true, y_pred)
    f1_score = sklearn.metrics.f1_score(y_true, y_pred, average="macro")
    
    print(f"Accuracy score: {accuracy_score}")
    print(f"Balanced accuracy score: {balanced_accuracy_score}")
    print(f"F1 score: {f1_score}")

In [74]:
print_metrics(y_test, predicted)

Accuracy score: 0.7763769077637691
Balanced accuracy score: 0.6210705492163321
F1 score: 0.6098491773272356
